In [1]:
#!pip3 install import_ipynb
#!pip3 install mpl_finance

In [4]:
#%run get_data.ipynb as get

import import_ipynb
import get_data as get

importing Jupyter notebook from get_data.ipynb


In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates 
from mpl_finance import candlestick_ohlc
import pandas_datareader.data as web
import os

In [8]:
def rsiFunc(ticker,n=14):

    '''This calculates the RS and RSI values 
    and automatically adds it to the ticker.csv 
    with the inbuilt get.add_col function defined before'''
    
    if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
        get.get_data(ticker, verbose = False);
    
    df_close = pd.read_csv('stock_dfs/{}.csv'.format(ticker)
                           ,parse_dates=True,index_col=0)['Adj Close']
    
    df = pd.DataFrame()
    deltas = np.diff(df_close)
    seed = deltas[:n]

    gain = seed[seed>=0]
    loss = -seed[seed<0]

    avg_gain = sum(gain)/n
    avg_loss = sum(loss)/n

    rs = avg_gain/avg_loss
    rs_list = np.zeros_like(df_close)
    rs_list[:n] = rs

    rsi = np.zeros_like(df_close)
    rsi[:n] = 100 - 100/(1+rs)

    for i in range(n,len(df_close)):
        delta = deltas[i-1]

        if delta > 0:
            upval = delta
            downval = 0
        else:
            upval = 0
            downval = -delta

        avg_gain = (avg_gain*(n-1)+upval)/n
        avg_loss = (avg_loss*(n-1)+downval)/n

        rs = avg_gain/avg_loss
        rs_list[i] = rs
        rsi[i] = 100 - 100/(1+rs)

    rs_list = rs_list[n:]
    rsi[:n-1] = np.nan
    
    df['RSI'] = rsi
    get.add_col(df,ticker,heading = 'RSI')

    return df

if __name__=="__main__":
    print(rsiFunc('TWTR',n=10).tail())

            RSI
1317  60.401729
1318  62.611250
1319  60.771216
1320  39.916463
1321  36.632360
